# Estudio de las reclasificaciones de variantes de KCNQ2 en ClinVar

Las reclasificaciones de las variantes presentes en [ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/) se
analizaron llevando a cabo el procedimiento facilitado por [Andrew Sharo](https://www.sciencedirect.com/science/article/pii/S0002929721000872?via%3Dihub).

Para este fin, es necesario acceder y conocer los archivos [FTP de ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/docs/ftp_primer/). Dentro de estos, se encuentran los **archivos
VCF** que contienen una instantánea de todas las variantes existentes en ClinVar y su clasificación en una
fecha específica. Se emplea el genoma humano GRCh38 y los años 2019-2021 para el estudio

___ 

**Contenido**
* [Apartado 1. Descarga y preprocesado de los archivos VCF](#ap1)
* [Apartado 2. Comparación de las variantes de KCNQ2 entre 2019 y 2021](#ap2)
* [Apartado 3. Visualización de las reclasificaciones en ClinVar](#ap3)


___

In [1]:
import io
import os
import pandas as pd

# FloWeaver for visualization
from ipysankeywidget import SankeyWidget
from floweaver import *

<br>
<br>

### Apartado 1. Descarga y preprocesado de los archivos VCF <a id="ap1"></a>

#### <u>Carga de los datos</u>

Los archivos VCF correspondientes a 2019 y 2021 se descargan, respectivamente, de:

* [Año 2019](https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/archive_2.0/2019/clinvar_20191219.vcf.gz).
* [Año 2021](https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/archive_2.0/2021/clinvar_20211204.vcf.gz).

Y se analizan de la siguiente manera siguiendo el procedimiento explicado en [Alistair Miles](http://alimanfoo.github.io/2017/06/14/read-vcf.html).

In [2]:
# Create a function to read a VCF file
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

In [3]:
# Load VCF files for 2019 and 2021
v19 = read_vcf("./Clinvar_2019.vcf")
v21 = read_vcf("./Clinvar_2021.vcf")

In [4]:
v21.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,1,925952,1019397,G,A,.,.,ALLELEID=1003021;CLNDISDB=MedGen:CN517202;CLND...
1,1,930139,1125147,C,T,.,.,ALLELEID=1110865;CLNDISDB=MedGen:CN517202;CLND...
2,1,930165,1164676,G,A,.,.,ALLELEID=1153701;CLNDISDB=MedGen:CN517202;CLND...
3,1,930187,1144630,C,T,.,.,ALLELEID=1131738;CLNDISDB=MedGen:CN517202;CLND...
4,1,930188,846933,G,A,.,.,AF_EXAC=0.00000;ALLELEID=824438;CLNDISDB=MedGe...


In [5]:
v21["INFO"][0]

'ALLELEID=1003021;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.11:g.925952G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=SAMD11:148398;MC=SO:0001583|missense_variant;ORIGIN=1;RS=1640863258'

<br>
<br>

#### <u> Filtrado de las variantes en KCNQ2</u>

De todas las variantes existentes en un fichero VCF, se filtran aquellas correspondientes al gen KCNQ2. Con ese fin, se filtra la columna de información `["INFO"]` a partir del campo `["GENEID"]`. En el caso concreto del gen KCNQ2, su `["GENEID"] = "KCNQ2:3785"`.

In [6]:
# Create a list to save variants INFO for KCNQ2
l21 = []

for i in v21["INFO"]:
    x = i.split(";")
    for j in x:
        if j[0:19] == "GENEINFO=KCNQ2:3785": # Filter variants
            l21.append(i) # Save ["INFO"] of matches to filter rows

# Create a new list to append specific info
l21_2 = []
for i in l21:
    l21_2.append(v21.loc[v21["INFO"] == i].values[:])
    

# Repeat for 2019
l19 = []

for i in v19["INFO"]:
    x = i.split(";")
    for j in x:
        if j[0:19] == "GENEINFO=KCNQ2:3785":
            l19.append(i)
            
l19_2 = []
for i in l19:
    l19_2.append(v19.loc[v19["INFO"] == i].values[:])

In [7]:
print("Total KCNQ2 variants in 2021:", len(l21))
print("Total KCNQ2 variants in 2019:", len(l19))

Total KCNQ2 variants in 2021: 1247
Total KCNQ2 variants in 2019: 792


In [8]:
print(l21[0])

AF_TGP=0.04752;ALLELEID=351405;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000020.11:g.63406349G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Benign;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=KCNQ2:3785;MC=SO:0001624|3_prime_UTR_variant;ORIGIN=1;RS=34690549


In [9]:
print(l21[0][0][0])
print(l21[0][0])

A
A


A continuación, se crean dos DataFrames (`v21_clean` y `v19_clean`) con la información de interés:

In [10]:
### 2021
# Create lists for fields of interest
CHROM = []
POS = []
ID = []
REF = []
ALT = []
QUAL = []
FILTER = []
INFO = []

for i in range(len(l21_2)):
    CHROM.append(l21_2[i][0][0])
    POS.append(l21_2[i][0][1])
    ID.append(l21_2[i][0][2])
    REF.append(l21_2[i][0][3])
    ALT.append(l21_2[i][0][4])
    QUAL.append(l21_2[i][0][5])
    FILTER.append(l21_2[i][0][6])
    INFO.append(l21_2[i][0][7])
    

# Build a Dataframe for 2021 KCNQ2 variants    
v21_clean = pd.DataFrame(CHROM, columns = ["CHROM"])
v21_clean["POS"] = POS
v21_clean["ID"] = ID
v21_clean["REF"] = REF
v21_clean["ALT"] = ALT
v21_clean["QUAL"] = QUAL
v21_clean["FILTER"] = FILTER
v21_clean["INFO"] = INFO

In [11]:
### 2019
# Create lists for fields of interest
CHROM2 = []
POS2 = []
ID2 = []
REF2 = []
ALT2 = []
QUAL2 = []
FILTER2 = []
INFO2 = []

for i in range(len(l19_2)):
    CHROM2.append(l19_2[i][0][0])
    POS2.append(l19_2[i][0][1])
    ID2.append(l19_2[i][0][2])
    REF2.append(l19_2[i][0][3])
    ALT2.append(l19_2[i][0][4])
    QUAL2.append(l19_2[i][0][5])
    FILTER2.append(l19_2[i][0][6])
    INFO2.append(l19_2[i][0][7])

# Build a Dataframe for 2019 KCNQ2 variants    
v19_clean = pd.DataFrame(CHROM2, columns = ["CHROM"])
v19_clean["POS"] = POS2
v19_clean["ID"] = ID2
v19_clean["REF"] = REF2
v19_clean["ALT"] = ALT2
v19_clean["QUAL"] = QUAL2
v19_clean["FILTER"] = FILTER2
v19_clean["INFO"] = INFO2

A continuación, se muestran los primeros 5 registros del DataFrame `v21_clean` creado:

In [12]:
v21_clean.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,20,63406349,339325,G,A,.,.,AF_TGP=0.04752;ALLELEID=351405;CLNDISDB=MedGen...
1,20,63406396,1259970,G,A,.,.,AF_TGP=0.10363;ALLELEID=336279;CLNDISDB=MedGen...
2,20,63406615,1260705,G,A,.,.,ALLELEID=1253193;CLNDISDB=MedGen:CN517202;CLND...
3,20,63406621,1252361,G,T,.,.,ALLELEID=1242307;CLNDISDB=MedGen:CN517202;CLND...
4,20,63406638,1214389,G,A,.,.,ALLELEID=1204374;CLNDISDB=MedGen:CN517202;CLND...


In [13]:
v21["INFO"][0]

'ALLELEID=1003021;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.11:g.925952G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=SAMD11:148398;MC=SO:0001583|missense_variant;ORIGIN=1;RS=1640863258'

Ejecutando las siguientes líneas de código se pueden guardar como ficheros CSV ambos DataFrames:

In [14]:
## - To CSV
#v21_clean.to_csv('KCNQ2_ClinVar_variants_2021.csv', sep=',', index=False)
#v19_clean.to_csv('KCNQ2_ClinVar_variants_2019.csv', sep=',', index=False)

<br>
<br>

#### <u> Incorporación de los datos clínicos de las variantes filtradas </u>

Para incorporar los datos clínicos se va a crear una nueva columna en los DataFrames correspondientes:

In [15]:
## FOR 2021
clin_21 = []

for i in v21_clean["INFO"]:
    x = i.split(";")
    for j in x:
        if j[0:7] == "CLNSIG=":
            j = j.split("=")[1]
            clin_21.append(j)
            
# Create that column in v21_clean
v21_clean["CLNSIG_21"] = clin_21



## FOR 2019
clin_19 = []

for i in v19_clean["INFO"]:
    x = i.split(";")
    for j in x:
        if j[0:7] == "CLNSIG=":
            j = j.split("=")[1]
            clin_19.append(j)
            
# Create that column in v19_clean
v19_clean["CLNSIG_19"] = clin_19

Además, se realiza un mappeo para corregir las claves únicas según el interés propio:

In [16]:
# Value to change : new value
clin_map = {
    'Benign/Likely_benign': 'Likely_benign',
    'Conflicting_interpretations_of_pathogenicity': 'Conflictive_interpretation',
    'Uncertain_significance': 'VUS',
    'Pathogenic/Likely_pathogenic': 'Likely_pathogenic'
}

# Use mapping to replace values
v21_clean["CLNSIG_21"].replace(clin_map, inplace=True)
v19_clean["CLNSIG_19"].replace(clin_map, inplace=True)

In [17]:
v21_clean.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,CLNSIG_21
0,20,63406349,339325,G,A,.,.,AF_TGP=0.04752;ALLELEID=351405;CLNDISDB=MedGen...,Benign
1,20,63406396,1259970,G,A,.,.,AF_TGP=0.10363;ALLELEID=336279;CLNDISDB=MedGen...,Benign
2,20,63406615,1260705,G,A,.,.,ALLELEID=1253193;CLNDISDB=MedGen:CN517202;CLND...,Benign
3,20,63406621,1252361,G,T,.,.,ALLELEID=1242307;CLNDISDB=MedGen:CN517202;CLND...,Benign
4,20,63406638,1214389,G,A,.,.,ALLELEID=1204374;CLNDISDB=MedGen:CN517202;CLND...,Likely_benign


<br>
<br>

### Apartado 2. Comparación de las variantes de KCNQ2 entre 2019 y 2021 <a id ="ap2"></a>

En este paso se comparan las variantes de KCNQ2 presentes en ClinVar entre los años 2019 y 2021. Las columnas de los DataFrames `"POS"`, `"ID"`, `"REF"` y `"ALT"` se emplean para este fin. Un nuevo DataFrame (`common`) se crea a partid de las mutaciones compartidas entre 2019 y 2021.

In [18]:
# Create a new DataFrame merging v21_clean and v19_clean
# Using columns of interest
common = pd.merge(v21_clean, v19_clean, how="inner", on=["CHROM", "POS", "REF", "ALT"])

# Clean useless columns
useless_col = ["ID_x", "QUAL_x", "FILTER_x", "INFO_x", "ID_y", "QUAL_y", "FILTER_y", "INFO_y"]
for i in useless_col:
    del common[i]

La siguiente línea de código permite almacenar en un fichero CSV el anterior DataFrame creado:

In [19]:
## - To CSV
#common.to_csv('common_variants_2021_2019.csv', sep=',', index=False)

In [20]:
common.head()

,CHROM,POS,REF,ALT,CLNSIG_21,CLNSIG_19
0,20,63406349,G,A,Benign,Benign
1,20,63406396,G,A,Benign,Benign
2,20,63406640,C,T,Likely_benign,Likely_benign
3,20,63406650,C,A,Likely_benign,Likely_benign
4,20,63406653,G,GGGCCC,Pathogenic,Pathogenic


In [21]:
common.shape

(784, 6)

A continuación, se estudian los "cambios" y "no cambios" de la clínica de las variantes de KCNQ2 en los años 2019 y 2021. El DataFrame `changed` almacena las variantes de KCNQ2 cuya clínica ha cambiado entre los años 2019 y 2021. El DataFrame `not_changed` almacena las variantes de KCNQ2 cuya clínica se ha mantenido entre los años 2019 y 2021.

In [22]:
# Selecting rows based on condition
changed = common.loc[common["CLNSIG_21"] != common["CLNSIG_19"]]
not_changed = common.loc[common["CLNSIG_21"] == common["CLNSIG_19"]]

In [23]:
print("La clínica ha cambiado en {} mutaciones".format(changed.shape[0]))
print("La clínica se ha mantenido en {} mutaciones".format(not_changed.shape[0]))

La clínica ha cambiado en 77 mutaciones
La clínica se ha mantenido en 707 mutaciones


Se comprueban los valores únicos existentes en la columna que almacena la clínica:

In [24]:
changed["CLNSIG_21"].unique() 

array(['Conflictive_interpretation', 'Benign', 'Likely_benign',
       'Pathogenic', 'VUS', 'Likely_pathogenic'], dtype=object)

Se realiza un conteo de los cambios entre las variantes de KCNQ2 en 2019 y 2021:

In [25]:
changed.groupby(['CLNSIG_19','CLNSIG_21']).size()

CLNSIG_19                   CLNSIG_21                 
Benign                      Conflictive_interpretation     1
                            Likely_benign                  2
Conflictive_interpretation  Benign                         1
                            Likely_benign                  5
                            Likely_pathogenic              4
                            Pathogenic                     3
                            VUS                            1
Likely_benign               Benign                        10
                            Conflictive_interpretation     4
Likely_pathogenic           Conflictive_interpretation    11
                            Pathogenic                     3
Pathogenic                  Benign                         2
                            Conflictive_interpretation     1
                            Likely_pathogenic             10
                            VUS                            2
VUS                         Co

<br>
<br>

### Apartado 3. Visualización de las reclasificaciones en ClinVar <a id="ap3"></a>

Para la representación de las reclasificaciones de KCNQ2 entre 2019 y 2021 se emplea la librería de [floWeaver](https://sankeyview.readthedocs.io/en/latest/tutorials/quickstart.html). Con este fin, se crea un DataFrame original según las especificaciones de floWeaver. Para eso, es necesario crear un fichero .csv con el siguiente formato:

* **Columna 1**: `CLINSIG_19`, es decir, la clínica de las variantes de KCNQ2 en 2019.
* **Columna 2**: `CLINSIG_21`, es decir, la clínica de las variantes de KCNQ2 en 2021.
* **Columna 3**: la clínica inicial, en este caso de 2019, con un código asignado (aunque no es obligatorio).
* **Columna 4**: el recuento de cuántas veces ocurre este cambio, obtenido a partir de la línea de código anterior.

In [26]:
# Load file
res = pd.read_csv("KCNQ2_reclass_changed_results.csv")
res

,CLNSIG_19,CLNSIG_21,Change_from,Count
0,Benign 2019,Conflictive interpretation 2021,B,1
1,Benign 2019,Likely benign 2021,B,2
2,Conflictive interpretation 2019,Benign 2021,C,1
3,Conflictive interpretation 2019,Likely benign 2021,C,5
4,Conflictive interpretation 2019,Likely pathogenic 2021,C,4
5,Conflictive interpretation 2019,Pathogenic 2021,C,3
6,Conflictive interpretation 2019,VUS 2021,C,1
7,Likely benign 2019,Benign 2021,LB,10
8,Likely benign 2019,Conflictive interpretation 2021,LB,4
9,Likely pathogenic 2019,Conflictive interpretation 2021,LP,11


Por último, se crea la representación de los cambios mediante un **diagrama de Sankey**:

In [27]:
# Set the default size to fit the documentation better.
size = dict(width=570, height=300)

# Need to rename the columns to make floWeaver work.
res.columns = ["source", "target", "type", "value"]

SankeyWidget(links=res.to_dict('records'))

SankeyWidget(links=[{'source': 'Benign 2019', 'target': 'Conflictive interpretation 2021', 'type': 'B', 'value…